In [103]:
import pandas as pd
import numpy as np
from pathlib import Path
import sklearn.neighbors
import sys
# these paths probably belong in a settings file or simillar. 
# Also try and stick to import first then executable code
sys.path.append('C:/Users/petera/Documents/Envirodual/paths')
sys.path.append('C:/Users/petera/Documents/Envirodual/values')
sys.path.append('C:/Users/petera/Documents/Envirodual/utils')
from path_definition import all_cities, city_paths, model
from preprocessing import one_hot_encode_column, decode_column
from values import OB_MID_dict, area_use_codes, area_use_columns, heated_area_codes
import warnings
warnings.filterwarnings('ignore')

In [104]:
city_name = 'ljubljana'
OB_MID = OB_MID_dict[city_name]
paths = city_paths[city_name]

## Read REN data

In [105]:
# columns to load from tables
cols_stavbe = ['STA_SID',
               'OB_MID',
               'ST_ETAZ',
               'ST_PRIT_ETAZE',
               'ST_STANOVANJ',
               'ST_POSLOVNIH_PROSTOROV',
               'DEJANSKA_RABA',
               'ID_TIP_STAVBE',
               'LETO_IZG_STA',
               'LETO_OBN_STREHE',
               'LETO_OBN_FASADE',
               'ID_KONSTRUKCIJE',
               'ID_OGREVANJE']

In [106]:
# read stavbe
stavbe_path = all_cities['root'] / all_cities['stavbe']
stavbe = pd.read_csv(
    stavbe_path,
    usecols=cols_stavbe,
    index_col='STA_SID',
    # encoding='cp1250',
    sep=';'
)

# keep only current municipality
stavbe = stavbe[stavbe.OB_MID.eq(OB_MID)]

In [107]:
cols_delistavb = ['STA_SID',
                  'DEJANSKA_RABA',
                  'UPOR_POV_STAN',
                  'NETO_TLORIS_POV_DST',
                  'LETO_OBN_OKEN',
                  'ID_POCIT_RABA']

# read delistavb
delistavb_path = all_cities['root'] / all_cities['delistavb']
delistavb = pd.read_csv(
    delistavb_path,
    usecols=cols_delistavb,
    # encoding='cp1250',
    decimal=',',
    sep=';'
)
delistavb = delistavb[delistavb.STA_SID.isin(stavbe.index.unique())]

In [108]:
# read sifranti
sifranti_path = all_cities['root'] / all_cities['sifranti']
sifranti = pd.read_csv(
    sifranti_path,
    # encoding='cp1250',
    sep=';'
)

# Temperature deficit

In [109]:
file_path_temp_deficit = all_cities['root'] / all_cities['temperature_deficit_cleaned']
temp_deficit = pd.read_csv(
    file_path_temp_deficit,
    index_col='STA_SID',
    usecols=['TEMP_DEFICIT','STA_SID']
)

# Read gas data

In [110]:
dtype = {'ZP': 'int', 'STA_SID': 'int'}
file_path_gas = all_cities['root'] / paths['gas_cleaned']
gas = pd.read_csv(
    file_path_gas,
    usecols=['ZP', 'STA_SID'],
    dtype=dtype,
    index_col='STA_SID'
)

In [111]:
df = gas.join(stavbe, how='inner').join(temp_deficit, how='left')

In [112]:
gas.shape, df.shape

((1958, 1), (1957, 14))

# Missing data REN

In [113]:
# drop entries where areas m2 not given
delistavb.fillna(0, inplace=True)

In [114]:
# collect total areas of building
df['UPORABNA_POVRSINA'] = delistavb.groupby('STA_SID')['UPOR_POV_STAN'].sum()
df['NETO_TLORIS'] = delistavb.groupby('STA_SID')['NETO_TLORIS_POV_DST'].sum()

In [115]:
# drop nans
df.dropna(subset=['UPORABNA_POVRSINA', 'NETO_TLORIS'], inplace=True)

In [116]:
# MISSING DATA
all_data_na = (df.isnull().sum() / len(df)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio
LETO_OBN_FASADE,71.640266
LETO_OBN_STREHE,51.047522
TEMP_DEFICIT,15.125192
ID_TIP_STAVBE,0.306592
ST_PRIT_ETAZE,0.306592
ID_OGREVANJE,0.204394
ST_ETAZ,0.204394
ID_KONSTRUKCIJE,0.153296
LETO_IZG_STA,0.102197
DEJANSKA_RABA,0.102197


In [117]:
# get mean windows age
df['LETO_OBN_OKEN'] = delistavb[delistavb.LETO_OBN_OKEN>0].groupby('STA_SID')['LETO_OBN_OKEN'].median().astype(int)

In [118]:
def fill_and_mark(df, column, value=0):
    '''fills missing values and creates new column marking which entries have been filled'''
    df_ = df.copy()
    df_['NA_'+column] = 0
    idx = df_[df_[column].isna()].index.astype('int32')
    df_.loc[idx,'NA_'+column] = 1
    df_[column].fillna(value, inplace=True)
    return df_

In [119]:
# If building rennovation year not given, set rennovation to construction year
v = df['LETO_IZG_STA'].median()

df = fill_and_mark(df, 'LETO_IZG_STA', v)
#df = fill_and_mark(df, 'LETO_OBN_OKEN', v)
#df = fill_and_mark(df, 'LETO_OBN_STREHE', v)
#df = fill_and_mark(df, 'LETO_OBN_FASADE', v)

In [120]:
for c in ['LETO_OBN_OKEN', 'LETO_OBN_STREHE', 'LETO_OBN_FASADE']:
    df['NA_'+c] = 0
    idx = (df[c].isna()) | (df[c]==0)
    df.loc[idx,'NA_'+c] = 1
    df.loc[idx, c] = df.loc[idx]['LETO_IZG_STA']

In [121]:
for c in ['ID_TIP_STAVBE', 'ID_OGREVANJE', 'ID_KONSTRUKCIJE', 'DEJANSKA_RABA', 'TEMP_DEFICIT']:
    v = df[c].median()
    df = fill_and_mark(df, c, v)

In [122]:
# Impute remaining missing values
def imputer_kNeighbors(df, attrib_to_impute, reference, n_neighbors=3):

    neigh = sklearn.neighbors.KNeighborsRegressor(n_neighbors=n_neighbors)

    y = df[df[attrib_to_impute] > 0][attrib_to_impute]
    x = df[df[attrib_to_impute] > 0][reference].values.reshape(-1, 1)

    neigh.fit(x, y)

    x_missing = df[df[attrib_to_impute].isna()][reference].values.reshape(-1, 1)
    imputed_values = neigh.predict(x_missing)
    # df.loc[df['attrib_to_impute'].isna(), 'attrib_to_impute'] = imputed_values
    return imputed_values

In [123]:
df.loc[df['ST_ETAZ'].isna(), 'ST_ETAZ'] = (
    imputer_kNeighbors(df, attrib_to_impute='ST_ETAZ', reference='NETO_TLORIS', n_neighbors=3)
)

df.loc[df['ST_PRIT_ETAZE'].isna(), 'ST_PRIT_ETAZE'] = (
    imputer_kNeighbors(df, attrib_to_impute='ST_PRIT_ETAZE', reference='NETO_TLORIS', n_neighbors=3)
)

In [124]:
all_data_na = (df.isnull().sum() / len(df)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio


## Engineer new variables

In [125]:
# add variable to represent the share of the area dedicated to second holiday houses/apartments
df['SHARE_HOLIDAY'] = delistavb[delistavb['ID_POCIT_RABA'] == 1622].groupby('STA_SID')['UPOR_POV_STAN'].sum()
df['SHARE_HOLIDAY'] = df['SHARE_HOLIDAY'] / df['UPORABNA_POVRSINA']
df['SHARE_HOLIDAY'].fillna(0, inplace=True)

In [126]:
columns_one_hot_encode = ['ID_TIP_STAVBE',
                          'DEJANSKA_RABA',
                          'ID_KONSTRUKCIJE',
                          'ID_OGREVANJE']

# decode columns
for col in columns_one_hot_encode:
    codes = df[col].unique()
    categories = sifranti.loc[sifranti.ID.isin(codes)]['IME'].values
    d = dict(zip(codes, categories))
    df = decode_column(df, col, d)

for c in columns_one_hot_encode:
    df = one_hot_encode_column(df, column=c)

# Create area use variables

In [127]:
area_use_cols = sifranti.loc[sifranti.POLJE_PK == 'DEJANSKA_RABA']['IME']
for c in area_use_cols:
    df[c] = 0

In [128]:
#areas_use = sifranti.loc[sifranti.POLJE_PK == 'DEJANSKA_RABA'][['ID', 'IME']]

# drop STA_SID that are not in the final table
delistavb = delistavb[delistavb.STA_SID.isin(df.index)]

In [129]:
for r, c in zip(area_use_codes, area_use_columns):
    tmp = delistavb.loc[delistavb.DEJANSKA_RABA == r][['STA_SID', 'UPOR_POV_STAN']]
    tmp = tmp.groupby('STA_SID')['UPOR_POV_STAN'].sum()
    df.loc[tmp.index, c] = tmp.values

In [130]:
# sum up heated area
ds = delistavb.loc[delistavb.DEJANSKA_RABA.isin(heated_area_codes)]
df['HEATED_AREA'] = ds.groupby('STA_SID')['UPOR_POV_STAN'].sum()
df.HEATED_AREA.fillna(0, inplace=True)

# Save result

In [131]:
all_data_na = (df.isnull().sum() / len(df)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio


In [132]:
file_path = all_cities['root'] / model['gas_' + city_name]
df.to_csv(
    file_path,
)

In [133]:
d = df['ZP']/df['UPORABNA_POVRSINA']

In [134]:
d[(d>5) & (d<1300)].mean() # Ljubljana 158.64 kWh

131.46343655939418

In [135]:
df.UPORABNA_POVRSINA.sum() # 7 752 556.6

708843.6

In [5]:
df.head(10).T

NameError: name 'df' is not defined

In [4]:
df.NA_ID_OGREVANJE.any()

NameError: name 'df' is not defined